In [ ]:
from typing import List, Union
import random
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import torch
import transformers
from peft import PeftModel
import torch
from transformers import GenerationConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
import json
from utils.callbacks import Iteratorize, Stream
from utils.prompter import Prompter
import gc
import pdb
import pandas as pd
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm
import numpy as np 
import copy
import evaluate
from rouge_score import rouge_scorer

            

### lode model

In [2]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
class EclmPretrained():
    def __init__(self):
        random.seed(10000)
        model_path = 'deanpp/E-Commerce_LLM'
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path,  device_map='auto',torch_dtype=torch.float16, trust_remote_code=True, do_sample=True)
        # load_in_4bit=True, 


model = EclmPretrained()

tokenizer = model.tokenizer
model = model.model
model.eval()
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer = tokenizer, 
    device_map="auto",
#     max_length=700,
)

### infer

In [4]:
model.save_pretrained('phi3-4k-r8')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'do_sample': True}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


In [13]:
pipe.tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"
def gen_res(system_prompt, instructions, is_do_sample):

    all_prompt_list_1 = []
    for instruction in instructions:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": instruction} ,
        ]
        all_prompt_list_1.append(messages)
    batch_size = 8
    generate_input = {
        "max_new_tokens":100,
        "do_sample":is_do_sample,
        "top_k":50,
        "top_p":0.95,
        "temperature":0.3,
        "batch_size":batch_size,
    }
    all_output = []
    for i in tqdm(list(range(0, len(all_prompt_list_1), batch_size))):
        batch_prompts = all_prompt_list_1[i:i + batch_size]
        batch_results = [i[0]['generated_text'][-1]['content'] for i in pipe(batch_prompts, **generate_input)]
        print(batch_results[0])
        all_output += batch_results
    torch.cuda.empty_cache()
    return all_output



### load data

In [14]:
all_dev_data = pd.read_csv("all_dev_data.csv")
rank_data = all_dev_data[all_dev_data['task'] == "query_product_rank_task"]
rank_label_data = pd.read_csv("query_product_rank_data.csv")
rank_data = pd.merge(rank_data, rank_label_data.rename(columns = {"prompt":"instruction"}), on = "instruction", how = "left")
rank_data['output_field'] = rank_data['output_field'].apply(lambda x:eval(x))
all_dev_data['task'].unique()

array(['sentiment_task_data', 'query_gen', 'is_buy_view_task',
       'buy_together_task', 'next_title_gen_mcer_task',
       'session_rec_task', 'next_title_gen_amazon_task', 'pair_rel_task',
       'answerability_task', 'query_product_rank_task',
       'query_product_rel_task', 'answer_generation_task'], dtype=object)

In [6]:
all_dev_data

,instruction,output,task
0,Instructions: Evaluate the following product r...,5,sentiment_task_data
1,A user on an online shopping website has just ...,men's rain jacket,query_gen
2,"Given the title of two products, predict if th...",1,is_buy_view_task
3,A user on an online shopping website has just ...,6,buy_together_task
4,Instructions: Evaluate the following product r...,4,sentiment_task_data
...,...,...,...
11354,Instructions: Evaluate the following product r...,5,sentiment_task_data
11355,Instructions: Evaluate the following product r...,5,sentiment_task_data
11356,Which of the following statements best describ...,3,pair_rel_task
11357,Which of the following statements best describ...,2,pair_rel_task


### eval ranking

In [15]:
def _parse_ranking(response: str) -> list:

    default_respomse = []
    # Keep only numeric characters and specific punctuation.
    cleaned_response = "".join(
        c for c in response if c.isnumeric() or c in [",", " "]
    )

    # Convert to list of integers
    ranked_items = []
    for item in cleaned_response.split(","):
        try:
            # Attempt to convert each item to an integer and add it to the list.
            int_item = int(item)
            if int_item <= 5:  # we know int_item can be at most 5
                ranked_items.append(int_item)
        except ValueError:
            pass  # Skip non-numeric items.

    # Consider only the first 5 unique elements.
    ranked_items = ranked_items[:5]

    # If there are duplicates, empty the list
    if len(ranked_items) != len(set(ranked_items)):
        ranked_items = default_respomse
    return ranked_items

def calculate_ndcg(predicted_relevance_scores: List[int], true_relevance_weights: List[float]) -> float:

    if len(predicted_relevance_scores) > len(true_relevance_weights):
        predicted_relevance_scores = predicted_relevance_scores[:len(true_relevance_weights)]

    dcg, idcg = 0.0, 0.0

    # Calculate DCG for the predicted ranking
    for i, score_index in enumerate(predicted_relevance_scores, start=1):
        if score_index - 1 < len(true_relevance_weights):
            relevance = true_relevance_weights[score_index - 1]
        else:
            relevance = 0
        dcg += (np.power(2, relevance) - 1) / np.log2(i + 1)
    
    # Calculate IDCG using sorted true relevance weights
    for i, weight in enumerate(sorted(true_relevance_weights, reverse=True), start=1):
        idcg += (np.power(2, weight) - 1) / np.log2(i + 1)
    
    # Avoid division by zero
    return 0 if idcg == 0 else dcg / idcg

system_prompt_1 = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. "
system_prompt_2 = "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions.\n"
system_prompt_3 = "You are an expert actor that can fully immerse yourself into any role given. You do not break character for any reason, even if someone tries addressing you as an AI or language model. Currently your role is shopping assistant, which is described in detail below. As a shopping assistant, continue the exchange with customer.\n"
system_prompt_4 = ""
all_output_rank = gen_res(system_prompt_1, rank_data['instruction'], is_do_sample = False)

all_output_rank_parse = [_parse_ranking(i) for i in all_output_rank]
rank_score_list = [calculate_ndcg(i,j) for i,j in zip(all_output_rank_parse, rank_data['output_field'])]
final_score = sum(rank_score_list) / len(rank_score_list)

  0%|                                                                                                                                                                            | 0/70 [00:00<?, ?it/s]/data/ptan499/python_packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/ptan499/python_packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|██▎                                                                                                                                      

5, 2, 1, 5, 3


  3%|████▋                                                                                                                                                               | 2/70 [00:02<01:28,  1.30s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


4, 2, 1, 5, 3


  4%|███████                                                                                                                                                             | 3/70 [00:03<01:09,  1.04s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

2, 3, 4, 1, 5


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 69/70 [00:53<00:00,  1.32it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


3, 4, 1, 5, 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [00:54<00:00,  1.30it/s]

2, 5, 1, 3, 4


In [16]:
final_score

0.7134521984245441

In [ ]:
final_score

In [9]:
final_score

0.810449331897452

In [ ]:
final_score

In [ ]:
final_score

In [ ]:
final_score

### sentiment

In [17]:
sentiment_data = all_dev_data[all_dev_data['task'] == 'sentiment_task_data']
sentiment_data['output'] = sentiment_data['output'].astype(int)
sentiment_data = copy.deepcopy(sentiment_data)
sentiment_data['length'] = sentiment_data['instruction'].apply(lambda x:len(x.split(" ")))
sentiment_data = sentiment_data.sort_values("length")

/data/ptan499/tmp/ipykernel_41457/3350053967.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_data['output'] = sentiment_data['output'].astype(int)


In [18]:
def _parse_multichoice(response: str) -> int:
    default_response = -1
    try:
        response = response.strip()
        return int(response[0])
    except Exception as e:
        print(f"SHOPBENCH_PARSER_WARNING::: Error parsing multichoice response: {e}. Responding with default : {default_response}")
        return default_response
def calculate_per_sample_accuracy(prediction: int, truth: int) -> bool:
    return prediction == truth
system_prompt_1 = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
system_prompt_2 = "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions.\n"
system_prompt_3 = "You are an expert actor that can fully immerse yourself into any role given. You do not break character for any reason, even if someone tries addressing you as an AI or language model. Currently your role is shopping assistant, which is described in detail below. As a shopping assistant, continue the exchange with customer.\n"
system_prompt_4 = ""
all_output_sentiment = gen_res(system_prompt_1, sentiment_data['instruction'], is_do_sample = False)

all_output_sentiment_parse = [_parse_multichoice(i) for i in all_output_sentiment]
sentiment_score_list = [calculate_per_sample_accuracy(i,j) for i,j in zip(all_output_sentiment_parse, sentiment_data['output'])]
final_score = sum(sentiment_score_list) / len(sentiment_score_list)

  0%|                                                                                                                                                                           | 0/121 [00:00<?, ?it/s]/data/ptan499/python_packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/ptan499/python_packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|█▎                                                                                                                                       

Based on the review text "Great!!", I would evaluate it as a 5, which is very positive. The reviewer's use of the word "Great" with two exclamation marks suggests a strong and enthusiastic endorsement of the product.


  2%|██▋                                                                                                                                                                | 2/121 [00:04<04:04,  2.05s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the review text "Crude Sucks", I would evaluate it as a 1, which is very negative. The words "Crude" and "Sucks" convey a strong sense of dislike and dissatisfaction with the product.


  2%|████                                                                                                                                                               | 3/121 [00:07<05:11,  2.64s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Based on the review, I would rate it a 5 out of 5. The reviewer is extremely positive about the book, praising the author's writing, the characters, and the story. They use superlatives such as "perfection personified", "real deal", and "honest from start to finish" to describe the book. They also mention that the author's writing makes them discover parts of themselves or relive emotions long forgotten, which is a high praise. The only negative


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 120/121 [07:18<00:04,  4.11s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the review, I would rate it a 3 out of 5. The reviewer seems to have mixed feelings about the game. On the one hand, they enjoyed the gameplay, particularly the controls, pitching, and base running. They also appreciated the graphics and the ease of navigating the menus. On the other hand, they were extremely disappointed with the lack of changes and improvements from the previous year's game, calling it a "roster update" with a different menu. They also criticized


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [07:22<00:00,  3.66s/it]

Based on the review, I would rate it a 2 out of 5. The reviewer seems to have mixed feelings about the game. On the one hand, they appreciate the challenging puzzles and the game's difficulty, which they consider a good challenge. On the other hand, they criticize the game's design, storyline, and acting, calling the acting "appalling" and the storyline "contrived". They also mention several technical issues with the CD version, including blurry graphics, poor object recognition


In [23]:
all_output_sentiment

['Based on the review text "Great!!", I would evaluate it as a 5, which is very positive. The reviewer\'s use of the word "Great" with two exclamation marks suggests a strong and enthusiastic endorsement of the product.',
 'Based on the review text "Cool", I would rate it a 4 out of 5. The word "Cool" is a positive adjective that suggests the reviewer is pleased with the product, but the brevity of the review and lack of specific details make it difficult to determine the extent of their satisfaction.',
 'Based on the input review text "Great Great", I would evaluate it as a 5, which is very positive. The reviewer has used the word "Great" twice, indicating a strong and enthusiastic endorsement of the product.',
 "Please provide the review text for the product, and I'll evaluate it on a scale of 1 to 5.",
 "Please provide the review text for the product, and I'll evaluate it on a scale of 1 to 5.",
 'Based on the input "nice", I would evaluate the product review as a 5, which is very p

In [ ]:
final_score

### pair relation

In [20]:
rel_data = all_dev_data[all_dev_data['task'] == 'pair_rel_task']
rel_data['output'] = rel_data['output'].astype(int)
rel_data = copy.deepcopy(rel_data)
rel_data['length'] = rel_data['instruction'].apply(lambda x:len(x.split(" ")))
rel_data = rel_data.sort_values("length")

/data/ptan499/tmp/ipykernel_41457/2955115628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_data['output'] = rel_data['output'].astype(int)


In [21]:
def _parse_multichoice(response: str) -> int:
    default_response = -1
    try:
        response = response.strip()
        return int(response[0])
    except Exception as e:
        print(f"SHOPBENCH_PARSER_WARNING::: Error parsing multichoice response: {e}. Responding with default : {default_response}")
        return default_response
def calculate_per_sample_accuracy(prediction: int, truth: int) -> bool:
    return prediction == truth
system_prompt_1 = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
system_prompt_2 = "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions.\n"
system_prompt_3 = "You are an expert actor that can fully immerse yourself into any role given. You do not break character for any reason, even if someone tries addressing you as an AI or language model. Currently your role is shopping assistant, which is described in detail below. As a shopping assistant, continue the exchange with customer.\n"
system_prompt_4 = ""
all_output_rel = gen_res(system_prompt_1, rel_data['instruction'], is_do_sample = False)

all_output_rel_parse = [_parse_multichoice(i) for i in all_output_rel]
rel_score_list = [calculate_per_sample_accuracy(i,j) for i,j in zip(all_output_rel_parse, rel_data['output'])]
final_score = sum(rel_score_list) / len(rel_score_list)

  0%|                                                                                                                                                                           | 0/116 [00:00<?, ?it/s]/data/ptan499/python_packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/ptan499/python_packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|█▍                                                                                                                                       

The correct answer is 1. irrelevant.

The query "license+plate+light+bulbs" appears to be related to automotive or lighting-related topics, whereas "enhoota" seems to be a random or nonsensical term. There is no apparent connection or relevance between the two queries, making them "irrelevant" to each other.


  2%|██▊                                                                                                                                                                | 2/116 [00:07<07:04,  3.72s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The correct answer is 2. narrowing.

The query "l trac" is likely referring to a specific type of camping equipment or gear, such as a lightweight backpacking trail. The query "camping" is a broader topic that encompasses many different aspects of outdoor recreation. Therefore, the relation between the two queries is one of narrowing, as the query "l trac" is a more specific search term that is related to the broader topic of camping.


  3%|████▏                                                                                                                                                              | 3/116 [00:10<06:44,  3.58s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The correct answer is 3. narrowing.

The query "colon clear formula" is likely a query that is used to filter or narrow down a set of results, such as a list of products or options. The query "I want a jacket without a hood that is fairly slim-fitting, but warm" is a specific request for a jacket that meets certain criteria.

The relation between the two queries is that the first query is used to narrow down the options to those that meet the criteria specified in


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 115/116 [06:58<00:03,  3.61s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The correct answer is 3. complement.

The query "refurbished laptop" is a specific query that provides more information about the type of laptop being searched for, while the query "I would be looking for a laptop that is good value and lightweight with long battery life" is a more general query that describes the desired features of the laptop. The two queries complement each other, as the first query provides a specific type of laptop to search for, and the second query provides a set of criteria


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 116/116 [07:02<00:00,  3.64s/it]

Based on the given instructions, I would say that the correct answer is:

2. irrelevant

The reason is that the two queries are unrelated in terms of their content, purpose, or meaning. The first query is about a jacket, while the second query is about a shirt with a slogan. There is no logical connection between the two, making them irrelevant to each other.


In [22]:
final_score

0.0

### buy_also

In [23]:
alsoBuy_data = copy.deepcopy(all_dev_data[all_dev_data['task'] == 'buy_together_task'])
alsoBuy_data['output'] = alsoBuy_data['output'].apply(lambda x:[int(i) for i in x.split(",")])
alsoBuy_data['length'] = alsoBuy_data['instruction'].apply(lambda x:len(x.split(" ")))
alsoBuy_data = alsoBuy_data.sort_values("length")

In [24]:
def _parse_retrieval(response: str) -> list:
    default_response = []
    try:
        cleaned_response = "".join(
            c for c in response if c.isnumeric() or c in [",", " "]
        )
        response = []
        for item in cleaned_response.split(","):
            try:
                response.append(int(item))
            except ValueError:
                pass  # Skip non-numeric items.
        return response
    except Exception as e:
        logger.warning(
            f"SHOPBENCH_PARSER_WARNING::: Error parsing retrieval response: {e}. Responding with default : {default_response}"
        )
        return default_response


def calculate_hit_rate_n(retrieved_int: List[int], truth: List[int]) -> float:
    hit = len(set(truth).intersection(set(retrieved_int[:len(truth)])))
    hit_rate = hit / len(truth)
    return hit_rate
system_prompt_1 = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
system_prompt_2 = "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions.\n"
system_prompt_3 = "You are an expert actor that can fully immerse yourself into any role given. You do not break character for any reason, even if someone tries addressing you as an AI or language model. Currently your role is shopping assistant, which is described in detail below. As a shopping assistant, continue the exchange with customer.\n"
system_prompt_4 = ""
all_output_alsoBuy = gen_res(system_prompt_1, alsoBuy_data['instruction'], is_do_sample = False)

all_output_alsoBuy_parse = [_parse_retrieval(i) for i in all_output_alsoBuy]
alsoBuy_score_list = [calculate_hit_rate_n(i,j) for i,j in zip(all_output_alsoBuy_parse, alsoBuy_data['output'])]
final_score = sum(alsoBuy_score_list) / len(alsoBuy_score_list)

  0%|                                                                                                                                                                           | 0/124 [00:00<?, ?it/s]/data/ptan499/python_packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/ptan499/python_packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|█▎                                                                                                                                       

2, 3, 8, 10, 13


  2%|██▋                                                                                                                                                                | 2/124 [00:01<01:36,  1.27it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


5, 6


  2%|███▉                                                                                                                                                               | 3/124 [00:02<01:35,  1.27it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

2, 4, 10, 11, 14


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 123/124 [01:46<00:00,  1.07it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2, 6


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [01:46<00:00,  1.16it/s]

1, 5, 6


In [25]:
final_score

0.6366125760649097

### sequential recommendation

In [21]:
set(all_dev_data['task'])

{'answer_generation_task',
 'answerability_task',
 'buy_together_task',
 'is_buy_view_task',
 'next_title_gen_amazon_task',
 'next_title_gen_mcer_task',
 'pair_rel_task',
 'query_gen',
 'query_product_rank_task',
 'query_product_rel_task',
 'sentiment_task_data',
 'session_rec_task'}

In [26]:
seq_data = copy.deepcopy(all_dev_data[all_dev_data['task'] == 'session_rec_task'])
seq_data['output'] = seq_data['output'].apply(lambda x:[int(i) for i in x.split(",")])
seq_data['length'] = seq_data['instruction'].apply(lambda x:len(x.split(" ")))
seq_data = seq_data.sort_values("length")

In [27]:
def _parse_retrieval(response: str) -> list:
    default_response = []
    try:
        cleaned_response = "".join(
            c for c in response if c.isnumeric() or c in [",", " "]
        )
        response = []
        for item in cleaned_response.split(","):
            try:
                response.append(int(item))
            except ValueError:
                pass  # Skip non-numeric items.
        return response
    except Exception as e:
        logger.warning(
            f"SHOPBENCH_PARSER_WARNING::: Error parsing retrieval response: {e}. Responding with default : {default_response}"
        )
        return default_response


def calculate_hit_rate_n(retrieved_int: List[int], truth: List[int]) -> float:
    hit = len(set(truth).intersection(set(retrieved_int[:len(truth)])))
    hit_rate = hit / len(truth)
    return hit_rate
system_prompt_1 = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
system_prompt_2 = "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions.\n"
system_prompt_3 = "You are an expert actor that can fully immerse yourself into any role given. You do not break character for any reason, even if someone tries addressing you as an AI or language model. Currently your role is shopping assistant, which is described in detail below. As a shopping assistant, continue the exchange with customer.\n"
system_prompt_4 = ""
all_output_seq = gen_res(system_prompt_1, seq_data['instruction'], is_do_sample = False)

all_output_seq_parse = [_parse_retrieval(i) for i in all_output_seq]
seq_score_list = [calculate_hit_rate_n(i,j) for i,j in zip(all_output_seq_parse, seq_data['output'])]
final_score = sum(seq_score_list) / len(seq_score_list)

  0%|                                                                                                                                                                           | 0/127 [00:00<?, ?it/s]/data/ptan499/python_packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/ptan499/python_packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|█▎                                                                                                                                       

1, 4, 6


  2%|██▌                                                                                                                                                                | 2/127 [00:00<00:58,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2, 7, 8


  2%|███▊                                                                                                                                                               | 3/127 [00:01<00:58,  2.12it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

1, 6, 8


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 126/127 [01:10<00:00,  1.44it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


6, 2, 5


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 127/127 [01:10<00:00,  1.79it/s]

1, 5, 7


In [28]:
final_score

0.5903534852989764

### answer generation

In [34]:
ans_data = copy.deepcopy(all_dev_data[all_dev_data['task'] == 'answer_generation_task'])
ans_data['length'] = ans_data['instruction'].apply(lambda x:len(x.split(" ")))
ans_data = ans_data.sort_values("length")

In [35]:
tmp_str = "Given a question and the related document, and generate the answer to the question based on the information provided in the document."
ans_data.apply(lambda x:x['instruction'].replace(tmp_str,"") + "#############" + x['output'],axis = 1).values[0:10]

array(["\nQuestion Text:How is it installed\nRelated Review:User0:works great posting from the laptop i installed it on\nUser1:I bought a samsung for my laptop, migrated and installed in snap.\nUser2:great hard drive! installed easily and haven't had a single issue since.\nUser3:Just installed this in my asus g750jw and now its lightnin fast but it sometimes freezes when i turn it on.\nUser4:I installed the 2 old 2Gb SIMMs from my MBPro into my 2010 iMac.\n\n### Answers: #############Just remove your current regular SATA disk, and plug this one in its place.",
       "\nQuestion Text:Are these true HEPA filters?\nRelated Review:User0:EXACTLY like the HEPA Pet Hair filters that come with the Neatos. Don't hesitate, just buy these! In fact, buy several.\nUser1:Not as good as the factory filters\nUser2:Much better filters than what came with my Neato\nUser3:Love my Neato vacuum and the filters fit just like the original.\nUser4:These are not Neato Brand filters but they still keep the dir

In [36]:
sacrebleu = None
def calculate_bleu_score(generated_text: str, reference_text: str, is_japanese: bool = False) -> float:
    global sacrebleu
    if sacrebleu is None:
        sacrebleu = evaluate.load("sacrebleu")

    # Preprocess input texts
    generated_text = generated_text.lstrip("\n").rstrip("\n").split("\n")[0]
    candidate = [generated_text]
    reference = [[reference_text]]

    # Compute BLEU score with or without Japanese-specific tokenization
    bleu_args = {"predictions": candidate, "references": reference, "lowercase": True}
    if is_japanese:
        bleu_args["tokenize"] = "ja-mecab"
    score = sacrebleu.compute(**bleu_args)["score"] / 100

    return score

def calculate_rougel(generation: str, truth: str) -> float:
    # Initialize the ROUGE scorer with the ROUGE-L metric
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    # Calculate the ROUGE scores between the generated text and the truth text
    scores = scorer.score(generation, truth)
    # Extract and return the ROUGE-L F-measure score
    return scores["rougeL"].fmeasure

In [ ]:

system_prompt_1 = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
# system_prompt_2 = "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions.\n"
# system_prompt_3 = "You are an expert actor that can fully immerse yourself into any role given. You do not break character for any reason, even if someone tries addressing you as an AI or language model. Currently your role is shopping assistant, which is described in detail below. As a shopping assistant, continue the exchange with customer.\n"
# system_prompt_4 = ""
all_output_ans = gen_res(system_prompt_1, ans_data['instruction'], is_do_sample = False)

all_output_ans_parse = all_output_ans
ans_score_bleu_list = [calculate_bleu_score(i,j) for i,j in zip(all_output_ans_parse, ans_data['output'])]
final_bleu_score = sum(ans_score_bleu_list) / len(ans_score_bleu_list)
## rouge_l
ans_score_rouge_list = [calculate_rougel(i,j) for i,j in zip(all_output_ans_parse, ans_data['output'])]
final_rouge_score = sum(ans_score_rouge_list) / len(ans_score_rouge_list)

  0%|                                                                                                                                                                           | 0/120 [00:00<?, ?it/s]/data/ptan499/python_packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/ptan499/python_packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|█▎                                                                                                                                       

Based on the related review, it can be inferred that the installation process is generally easy and straightforward. Here's a possible answer:

The installation process is easy and straightforward. Users have reported installing it on their laptops, including Samsung and Asus models, and have had no issues. One user even mentioned migrating and installing it in Snap. However, one user did experience some freezing issues when turning on their Asus laptop, but this may be a specific hardware-related problem. Overall, the installation process seems to be


  2%|██▋                                                                                                                                                                | 2/120 [00:07<07:06,  3.61s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the review, it is not explicitly stated whether the product is compatible with the Sony VPL-HW50ES projector. However, the reviewer User0 mentions that the product "fit perfect" and is "stable enough" for their "heavy Sony Pearl", which suggests that the product may be compatible with the Sony projector.


  2%|████                                                                                                                                                               | 3/120 [00:10<07:05,  3.64s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Based on the reviews provided, it seems that the product does not work on a Daisy 415. User0 mentions that their product, which looks similar to the "Daisy Electric Point Sight", does not have the same logo and does not work properly. User1 does not mention the specific model they used, but they mention that it's a cheap product and they got it for $8. User3 also mentions that their product sometimes turns on by itself and the off button doesn't work most of


 39%|███████████████████████████████████████████████████████████████▍                                                                                                  | 47/120 [02:55<04:35,  3.77s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the related review, the answer to the question "Compatible with Windows 8.1?" is:

Yes, according to User2, the device is compatible with Windows 8.1, as it was recognized immediately when plugged into their HP desktop running Windows 8.1.


 40%|████████████████████████████████████████████████████████████████▊                                                                                                 | 48/120 [02:58<04:30,  3.75s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the reviews provided, it can be inferred that the cuffs are medium to heavy resistance in the water. User0 mentions that they are "wonderful for resistance when water walking", which suggests a moderate level of resistance. User1 also mentions that they are "perfect for water aerobics in the deep end of water", which implies a level of resistance suitable for a more intense workout. The fact that User2 mentions that the foam inserts are still good after over 10 years, despite


In [ ]:
final_bleu_score

In [ ]:
final_rouge_score